In [7]:
# Imports
from matplotlib.collections import LineCollection
from matplotlib import cm
import math

import pickle
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches
from copy import deepcopy
#import plotly.graph_objects as go
from scipy.optimize import curve_fit
from dataclasses import make_dataclass
from collections import Counter

#from tqdm import tqdm
import pandas as pd

import sys
module_path = 'maze_utils' 
if module_path not in sys.path:
    sys.path.append(module_path)

from MM_Plot_Utils import plot, hist
from MM_Maze_Utils import *
from MM_Traj_Utils import *

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [8]:
# Define and populate the maze structure ma, used as an argument for many functions
ma=NewMaze(6)

# non-valid maze parts
re=[[-0.5,0.5,1,1],[-0.5,4.5,1,1],[-0.5,8.5,1,1],[-0.5,12.5,1,1],
       [2.5,13.5,1,1],[6.5,13.5,1,1],[10.5,13.5,1,1],
       [13.5,12.5,1,1],[13.5,8.5,1,1],[13.5,4.5,1,1],[13.5,0.5,1,1],
       [10.5,-0.5,1,1],[6.5,-0.5,1,1],[2.5,-0.5,1,1],
       [6.5,1.5,1,1],[6.5,11.5,1,1],[10.5,5.5,1,1],[10.5,7.5,1,1],
       [5.5,4.5,1,1],[5.5,8.5,1,1],[7.5,4.5,1,1],[7.5,8.5,1,1],[2.5,5.5,1,1],[2.5,7.5,1,1],
       [-0.5,2.5,3,1],[-0.5,10.5,3,1],[11.5,10.5,3,1],[11.5,2.5,3,1],[5.5,0.5,3,1],[5.5,12.5,3,1],
       [7.5,6.5,7,1]]

In [9]:
# removes consective duplicate coords from a path list of coordinates
def remove_consecutive_duplicate_coords(path):
    return [v for i, v in enumerate(path) if i == 0 or (path[i] != path[i-1]).any()]

In [10]:
# get a path of a bout (list of coordinates)
def get_path_552(nickname,b):
    tf=LoadTraj(nickname+'-tf')
    x=-0.5+15*tf.ke[b][:,0]
    y=-0.5+15*tf.ke[b][:,1]
    t=np.linspace(0,1,x.shape[0]) # time variable
    points_raw = np.array([list(x),list(y)]).transpose().tolist()
    points_no_nan = list(filter(lambda x: (not math.isnan(x[0])) or (not math.isnan(x[1])), points_raw))
    points_rounded = np.array(np.round(points_no_nan), dtype=int)
    points_valid = [x for x in points_rounded if tuple(x) in ma.ce.keys()]
    points_valid = remove_consecutive_duplicate_coords(points_valid)
    #points = [i for (i,j) in zip(points_valid[1:], points_valid) if len(ConnectPath(ma.ce[tuple(j)], ma.ce[tuple(i)], ma))<6]

    new_points = [np.array([0, 7])]
    for next_coord, curr_coord in zip(points_valid[1:], points_valid):
        path_length = len(ConnectPath(ma.ce[tuple(curr_coord)], ma.ce[tuple(next_coord)], ma))
        if path_length<4:
            new_points.append(next_coord)

    points_copy = new_points.copy()
    offset = 0
    for i, (x,y) in enumerate(zip(new_points[1:], new_points)):
        path = ConnectPath(ma.ce[tuple(y)], ma.ce[tuple(x)], ma)
        if len(path)>2:
            coords = []
            for idx in path[1:-1]:
                coords.append(np.array([ma.xc[idx], ma.yc[idx]]))
            points_copy[i+1+offset:i+1+offset] = coords
            offset += len(coords)

    return points_copy, t

In [11]:
# plot a path (list of coordinates) with path coloured based on time
def plot_path_552_time(points, t, ax):
    points = np.array(points).reshape(-1,1,2)

    segs=np.concatenate([points[:-1],points[1:]],axis=1) # array of segments
    lc=LineCollection(segs,cmap=plt.get_cmap('viridis'),linewidths=1.5) # collect segments
    lc.set_array(t) # color the segments by the time variable
    plot(ma.wa[:,0],ma.wa[:,1],fmts=['k-'],equal=True,linewidth=1,yflip=True,
              xhide=True,yhide=True,axes=ax) # Draw the maze outline    
    for r in re:
        rect=patches.Rectangle((r[0],r[1]),r[2],r[3],linewidth=1,edgecolor='lightgray',facecolor='lightgray')
        ax.add_patch(rect)
    ax.axis('off'); # turn off the axes
    ax.add_collection(lc); # add the collection to the plot

In [12]:
# plot a path (list of coordinates) without time
def plot_path_552(points, ax):
    points = np.array(points).reshape(-1,1,2)

    segs=np.concatenate([points[:-1],points[1:]],axis=1) # array of segments
    lc=LineCollection(segs,cmap=plt.get_cmap('viridis'),linewidths=1.5) # collect segments
    lc.set_array(np.linspace(0,1,points.shape[0])) # color the segments
    plot(ma.wa[:,0],ma.wa[:,1],fmts=['k-'],equal=True,linewidth=1,yflip=True,
              xhide=True,yhide=True,axes=ax) # Draw the maze outline    
    for r in re:
        rect=patches.Rectangle((r[0],r[1]),r[2],r[3],linewidth=1,edgecolor='lightgray',facecolor='lightgray')
        ax.add_patch(rect)
    ax.add_collection(lc); # add the collection to the plot
    ax.axis('off'); # turn off the axes

In [ ]:
# # get first proper bouts (beyond the first junction) of all 19 animals
# with open('outdata/HomeRuns', 'rb') as f:
#     Names,HR=pickle.load(f) # bout, starting node, node distance, absolute frame   
# sb=[hr[hr[:,2]>1,0][0] for hr in HR] # bout of first home run if past first intersection
# paths, time = [], []
# sb_dict = {}
# for i,nickname in tqdm(enumerate(Names), total=len(Names)):
#     sb_dict[nickname] = sb[i]
#     path, time_var = get_path_552(nickname,sb[i])
#     paths.append(path)
#     time.append(time_var)

In [ ]:
# # plot first proper bouts (beyond the first junction) of all 19 animals (with time)
# fig,ax=plt.subplots(4,5,figsize=(12,10))
# for i, path in tqdm(enumerate(paths), total=len(paths)):
#     plot_path_552_time(path, time[i], ax[i//5,i%5])

# lastax=ax[-1,-1] # deal with the last panel
# plot([0],[0],fmts=['w.'],equal=True,yflip=True,
#           xhide=True,yhide=True,axes=lastax) # Draw nothing, helps for some reason 
# lastax.axis('off')
# plt.show()

In [ ]:
# # plot first proper bouts (beyond the first junction) of all 19 animals (no time)
# fig,ax=plt.subplots(4,5,figsize=(12,10))
# for i, path in tqdm(enumerate(paths), total=len(paths)):
#     path_no_time = remove_consecutive_duplicate_coords(path)
#     plot_path_552(path_no_time, ax[i//5,i%5])

# lastax=ax[-1,-1] # deal with the last panel
# plot([0],[0],fmts=['w.'],equal=True,yflip=True,
#           xhide=True,yhide=True,axes=lastax) # Draw nothing, helps for some reason 
# lastax.axis('off')
# plt.show()

In [13]:
# this is a mess, but it works. Please don't judge :(
def steps_to_mouse_action(step_list, index_list):
    end_nodes = [idx_list[-1] for idx_list in ma.ru]
    actions = []
    for i, (next_step, curr_step) in enumerate(zip(step_list[1:], step_list)):
        if (next_step == 'node_in' and curr_step == 'node_in') or (next_step == 'node_out' and curr_step == 'node_out'):
            actions.append('straight')
        elif (next_step == 'node_in' and curr_step == 'node_out'):
            if index_list[i+1] in end_nodes:
                if index_list[i+1]>=112:
                    actions.append('dead end turn around')
                else:
                    actions.append('node turn around')
            else:
                actions.append('turn around')
        elif (next_step == 'node_out' and curr_step == 'node_in'):
            if index_list[i+1] in end_nodes:
                if index_list[i+1]>=112:
                    actions.append('dead end turn around')
                else:
                    actions.append('node turn around')
            else:
                actions.append('turn around')
        elif (next_step == 'in left' and curr_step == 'node_in'):
            actions.append('left turn')
        elif (next_step == 'in right' and curr_step == 'node_in'):
            actions.append('right turn')
        elif (next_step == 'in left' and curr_step == 'out left'):
            if index_list[i+1] in end_nodes:
                if index_list[i+1]>=112:
                    actions.append('dead end turn around')
                else:
                    actions.append('node turn around')
            else:
                actions.append('turn around')
        elif (next_step == 'in right' and curr_step == 'out right'):
            if index_list[i+1] in end_nodes:
                if index_list[i+1]>=112:
                    actions.append('dead end turn around')
                else:
                    actions.append('node turn around')
            else:
                actions.append('turn around')
        elif (next_step == 'out left' and curr_step == 'in left'):
            if index_list[i+1] in end_nodes:
                if index_list[i+1]>=112:
                    actions.append('dead end turn around')
                else:
                    actions.append('node turn around')
            else:
                actions.append('turn around')
        elif (next_step == 'out right' and curr_step == 'in right'):
            if index_list[i+1] in end_nodes:
                if index_list[i+1]>=112:
                    actions.append('dead end turn around')
                else:
                    actions.append('node turn around')
            else:
                actions.append('turn around')
        elif (next_step == 'in left' and curr_step == 'out right'):
            actions.append('straight through node')
        elif (next_step == 'in right' and curr_step == 'out left'):
            actions.append('straight through node')
        elif (next_step == 'node_in' and curr_step == 'in left'):
            actions.append('straight')
        elif (next_step == 'node_in' and curr_step == 'in right'):
            actions.append('straight')
        elif (next_step == 'out left' and curr_step == 'node_out'):
            actions.append('straight')
        elif (next_step == 'out right' and curr_step == 'node_out'):
            actions.append('straight')
        elif (next_step == 'node_out' and curr_step == 'out left'):
            actions.append('right turn')
        elif (next_step == 'node_out' and curr_step == 'out right'):
            actions.append('left turn')
        elif (next_step == 'in right' and curr_step == 'in left'):
            actions.append('right turn')
        elif (next_step == 'in left' and curr_step == 'in left'):
            actions.append('left turn')
        elif (next_step == 'in right' and curr_step == 'in right'):
            actions.append('right turn')
        elif (next_step == 'in left' and curr_step == 'in right'):
            actions.append('left turn')
        elif (next_step == 'out left' and curr_step == 'out left'):
            actions.append('right turn')
        elif (next_step == 'out right' and curr_step == 'out right'):
            actions.append('left turn')
        elif (next_step == 'out left' and curr_step == 'out right'):
            actions.append('right turn')
        elif (next_step == 'out right' and curr_step == 'out left'):
            actions.append('left turn')
        else:
            actions.append('NOT HANDLED')

    return actions

In [42]:
print(ma.ru)

[[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15], [16, 17, 18, 19], [20, 21, 22, 23], [24, 25, 26, 27], [28, 29, 30, 31], [32, 33], [34, 35], [36, 37], [38, 39], [40, 41], [42, 43], [44, 45], [46, 47], [48, 49], [50, 51], [52, 53], [54, 55], [56, 57], [58, 59], [60, 61], [62, 63], [64, 65], [66, 67], [68, 69], [70, 71], [72, 73], [74, 75], [76, 77], [78, 79], [80], [81], [82], [83], [84], [85], [86], [87], [88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99], [100], [101], [102], [103], [104], [105], [106], [107], [108], [109], [110], [111], [112], [113], [114], [115], [116], [117], [118], [119], [120], [121], [122], [123], [124], [125], [126], [127], [128], [129], [130], [131], [132], [133], [134], [135], [136], [137], [138], [139], [140], [141], [142], [143], [144], [145], [146], [147], [148], [149], [150], [151], [152], [153], [154], [155], [156], [157], [158], [159], [160], [161], [162], [163], [164], [165], [166], [167], [168], [169], [170], [171], [17

In [43]:
# gets a complete list of actions for a path
def get_path_actions(path):
    #convert coordinates to indices
    index_list = [ma.ce[tuple(coord)] for coord in path] 

    # convert indicies to nodes
    # maze.ru is a list of lists, where the inner lists are the indices of the coordinates that are associated to each node
    node_list= []
    for index in index_list:
        for i, node_idx_list in enumerate(ma.ru):
            if index in node_idx_list:
                node_list.append(i)
    
    # terminate path if mouse reaches node 116 (index 165)
    try:
        water_node= node_list.index(116)
        node_list= node_list[:water_node+1]
        water_reached= True
    #.index raises ValueError if value does not exist in list
    except ValueError:
        water_reached=False



    # convert nodes to steps
    step_list= []
    for i, (start, end) in enumerate(zip(node_list, node_list[1:])):
        if start == end:
            # travelling inwards along node
            if index_list[i+1] > index_list[i]:
                step_type = 'node_in'
            # travelling outwards along node
            elif index_list[i+1] < index_list[i]:
                step_type = 'node_out'
        else:
            # changing nodes
            step_type = ma.st[start,end]
            
        step_list.append(step_type)

    for i, step_num in enumerate(step_list):
        # in left = 0; in right = 1; out left = 2; out right = 3; illegal = -1
        if step_num == 0:
            step_list[i] = 'in left'
        elif step_num == 1:
            step_list[i] = 'in right'
        elif step_num == 2:
            step_list[i] = 'out left'
        elif step_num == 3:
            step_list[i] = 'out right'

    # convert steps to actions
    return steps_to_mouse_action(step_list, index_list), node_list, water_reached

In [56]:
def get_path_metrics_df(mouse,bout,actions, node_list,water_reached):
    # get total count of each action
    counts_dict = dict(sorted(dict(Counter(actions)).items()))

    corridor_actions = ['straight', 'turn around']
    node_actions = ['left turn', 'right turn', 'straight through node', 'node turn around']

    corridor_counts = {key: counts_dict[key] for key in counts_dict if key in corridor_actions}
    node_counts = {key: counts_dict[key] for key in counts_dict if key in node_actions}

    # get various counts of the number of actions performed
    total_num_actions = len(actions)
    num_actions_no_dead_ends = total_num_actions - counts_dict['dead end turn around']
    num_actions_corridor = sum([value for value in corridor_counts.values()])
    num_actions_node = sum([value for value in node_counts.values()])

    # get percentages of corridor and node actions
    corridor_percents = {key: round(value/num_actions_corridor*100,2) for key, value in corridor_counts.items()}
    node_percents = {key: round(value/num_actions_node*100,2) for key, value in node_counts.items()}

    # count how many consecutive same turns there are
    turns = [action for action in actions if action in ['left turn', 'right turn']]
    count = 1
    consecutive_same_turns = []
    for next_turn, curr_turn in zip(turns[1:], turns):
        if next_turn == curr_turn:
            count += 1
        else:
            consecutive_same_turns.append(count)
            count = 1
    same_turn_counts = dict(sorted(dict(Counter(consecutive_same_turns)).items()))
    same_turn_percents = {key: round(value*key/(len(turns)-1)*100,2) for key, value in same_turn_counts.items()}

    node_list_cleaned = [next_node for next_node, curr_node in zip(node_list[1:], node_list) if next_node != curr_node]
    node_list_cleaned.insert(0, node_list[0])
    num_nodes_visited = len(node_list_cleaned)
    node_counts = dict(sorted(dict(Counter(node_list_cleaned)).items()))

    # get number of nodes visited before reaching reward and how many times they visit each node
    if water_reached == True:
        # get first index for node 116 and remove everything after, then get counts
        water_node= node_list_cleaned.index(116)
        node_list_upto_water= node_list_cleaned[:water_node+1]
        num_nodes_visited_before_water = len(node_list_upto_water)
        node_counts_before_water = dict(sorted(dict(Counter(node_list_upto_water)).items()))
    elif water_reached == False:
        num_nodes_visited_before_water= np.nan
        node_counts_before_water= np.nan


    return pd.DataFrame({'nickname':mouse,'bout':bout,'total # of actions': total_num_actions, 
                         '# of actions w/o dead ends': num_actions_no_dead_ends,
                        '# of corridor actions': num_actions_corridor, 
                        '# of node actions': num_actions_node,
                        "% of each corridor action": [corridor_percents], 
                        "% of each node action": [node_percents],
                        "% of consecutive turn intervals": [same_turn_percents], 
                        "total # of unique nodes visisted": num_nodes_visited,
                        "# of visits for each visited node": [node_counts], 
                        "water_reached": water_reached,
                        "total # of unique nodes visisted before water": num_nodes_visited_before_water,
                        "# of visits for each visited node before water": [node_counts_before_water]})

In [57]:
# example of using the above functions
path, _ = get_path_552('B1', 13)
actions, node_list, water_reached = get_path_actions(path)
metrics_df = get_path_metrics_df('B1', 13, actions, node_list, water_reached)

In [58]:
metrics_df

,nickname,bout,total # of actions,# of actions w/o dead ends,# of corridor actions,# of node actions,% of each corridor action,% of each node action,% of consecutive turn intervals,total # of unique nodes visisted,# of visits for each visited node,water_reached,total # of unique nodes visisted before water,# of visits for each visited node before water
0,B1,13,147,138,78,60,"{'straight': 51.28, 'turn around': 48.72}","{'left turn': 20.0, 'node turn around': 35.0, ...","{1: 12.0, 2: 72.0, 3: 12.0}",81,"{0: 10, 1: 6, 2: 11, 5: 13, 6: 3, 11: 5, 12: 5...",False,NaN,NaN


# Next steps:
1. only get rewarded mice (10 of them)- done
2. get paths of all rewarded mice past first intersection (done below, but not yet separated by reward vs. non-reward)- done
3. cut the paths off if they reach index 165 or node 116, otherwise let the paths go until they exit- done
4. get metric for how many nodes they visit before reaching the reward- done
5. get metric for how many times they visit a node- done
6. Fix dead end turn around (key doesn't exist for a lot of them)


In [23]:
# find bouts for each mice where mice go past first intersection and return home without reversals
with open('outdata/HomeRuns', 'rb') as f:
    Names,HR=pickle.load(f) # bout, starting node, node distance, absolute frame   
sb=[hr[hr[:,2]>1,0] for hr in HR] # bout of first home run if past first intersection
bout_dict = {}
for mouse, sb_list in zip(Names, sb):
    bout_dict[mouse] = sb_list
display(bout_dict)

{'B1': array([ 13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  38,  39,  40,
         41,  42,  43,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
         55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
         68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
         81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  95,  97,  99,
        100, 101, 102, 103], dtype=int64),
 'B2': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,
         55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,
         68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
         81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  

In [59]:
import traceback
all_metrics=[]
RewNames=['B1','B2','B3','B4','C1','C3','C6','C7','C8','C9']
# loop through each rewarded mouse --> we care about the rewarded mice
for name in RewNames:
    print(name)
    tf=LoadTraj(f'{name}-tf')
    # loop through each bout
    for bout in range(len(tf.ke)):
        # if bout is a home run, get path -> we don't care about the other mice
        if bout in bout_dict[name]:
            print(bout)
            try:
                path, _ = get_path_552(name, bout)
                actions, node_list, water_reached = get_path_actions(path)
                metrics_df = get_path_metrics_df(mouse, bout,actions, node_list, water_reached)
                all_metrics.append(metrics_df)
            except Exception as e:
                #traceback.print_exc()
                print(e)
                print(f"something went wrong in mouse: {name}, bout: {bout} --> investigate l8er")
                pass
            
all_metrics_df= pd.concat(all_metrics, ignore_index=True)

B1
13
14
15
16
17
18
19
20
'dead end turn around'
something went wrong in mouse: B1, bout: 20 --> investigate l8er
21
22
23
25
26
27
28
29
30
31
32
'dead end turn around'
something went wrong in mouse: B1, bout: 32 --> investigate l8er
33
34
35
'dead end turn around'
something went wrong in mouse: B1, bout: 35 --> investigate l8er
36
38
39
'dead end turn around'
something went wrong in mouse: B1, bout: 39 --> investigate l8er
40
'dead end turn around'
something went wrong in mouse: B1, bout: 40 --> investigate l8er
41
42
'dead end turn around'
something went wrong in mouse: B1, bout: 42 --> investigate l8er
43
'dead end turn around'
something went wrong in mouse: B1, bout: 43 --> investigate l8er
45
'dead end turn around'
something went wrong in mouse: B1, bout: 45 --> investigate l8er
46
47
48
'dead end turn around'
something went wrong in mouse: B1, bout: 48 --> investigate l8er
49
'dead end turn around'
something went wrong in mouse: B1, bout: 49 --> investigate l8er
50
'dead end tu

In [55]:
display(all_metrics_df)
display(all_metrics_df.iloc[0]["# of visits for visited each node"])
display(all_metrics_df.iloc[10]["# of visits for visited each node"])
display(all_metrics_df.iloc[20]["# of visits for visited each node"])

,total # of actions,# of actions w/o dead ends,# of corridor actions,# of node actions,% of each corridor action,% of each node action,% of consecutive turn intervals,total # of unique nodes visisted,# of visits for each visited node,water_reached,total # of unique nodes visisted before water,# of visits for each visited node before water
0,147,138,78,60,"{'straight': 51.28, 'turn around': 48.72}","{'left turn': 20.0, 'node turn around': 35.0, ...","{1: 12.0, 2: 72.0, 3: 12.0}",81,"{0: 10, 1: 6, 2: 11, 5: 13, 6: 3, 11: 5, 12: 5...",False,NaN,NaN
1,209,184,76,108,"{'straight': 68.42, 'turn around': 31.58}","{'left turn': 33.33, 'node turn around': 12.96...","{1: 15.25, 2: 30.51, 3: 25.42, 4: 6.78, 5: 8.4...",141,"{0: 4, 1: 2, 2: 5, 5: 6, 6: 8, 11: 9, 12: 3, 1...",False,NaN,NaN
2,528,457,149,308,"{'straight': 71.14, 'turn around': 28.86}","{'left turn': 36.69, 'node turn around': 14.94...","{1: 17.3, 2: 25.95, 3: 17.84, 4: 15.14, 5: 16....",387,"{0: 5, 1: 4, 2: 2, 3: 1, 4: 19, 9: 31, 10: 28,...",False,NaN,NaN
3,642,543,151,392,"{'straight': 74.17, 'turn around': 25.83}","{'left turn': 30.61, 'node turn around': 18.88...","{1: 15.02, 2: 24.41, 3: 25.35, 4: 9.39, 5: 7.0...",489,"{0: 9, 1: 8, 2: 3, 3: 13, 4: 1, 5: 4, 7: 20, 8...",False,NaN,NaN
4,677,619,205,414,"{'straight': 65.37, 'turn around': 34.63}","{'left turn': 25.12, 'node turn around': 29.47...","{1: 21.57, 2: 24.51, 3: 14.71, 4: 25.49, 5: 4....",463,"{0: 3, 1: 8, 2: 2, 3: 10, 4: 13, 5: 7, 6: 2, 7...",True,463.0,"{0: 3, 1: 8, 2: 2, 3: 10, 4: 13, 5: 7, 6: 2, 7..."
...,...,...,...,...,...,...,...,...,...,...,...,...
533,82,73,35,38,"{'straight': 97.14, 'turn around': 2.86}","{'left turn': 39.47, 'node turn around': 13.16...","{1: 13.04, 2: 8.7, 3: 26.09, 4: 52.17}",49,"{0: 2, 1: 3, 2: 1, 3: 2, 4: 1, 6: 1, 8: 4, 13:...",True,49.0,"{0: 2, 1: 3, 2: 1, 3: 2, 4: 1, 6: 1, 8: 4, 13:..."
534,82,78,38,40,"{'straight': 94.74, 'turn around': 5.26}","{'left turn': 27.5, 'node turn around': 22.5, ...","{1: 33.33, 2: 22.22, 3: 11.11, 4: 14.81, 5: 18...",43,"{0: 2, 1: 2, 2: 1, 3: 2, 6: 1, 7: 5, 13: 1, 15...",True,43.0,"{0: 2, 1: 2, 2: 1, 3: 2, 6: 1, 7: 5, 13: 1, 15..."
535,74,68,37,31,"{'straight': 86.49, 'turn around': 13.51}","{'left turn': 35.48, 'node turn around': 12.9,...","{1: 38.1, 2: 38.1, 5: 23.81}",35,"{0: 2, 1: 2, 2: 1, 3: 2, 6: 1, 7: 2, 13: 1, 15...",True,35.0,"{0: 2, 1: 2, 2: 1, 3: 2, 6: 1, 7: 2, 13: 1, 15..."
536,146,124,49,75,"{'straight': 61.22, 'turn around': 38.78}","{'left turn': 25.33, 'node turn around': 22.67...","{1: 11.43, 2: 17.14, 3: 17.14, 4: 22.86, 5: 14...",103,"{0: 5, 1: 3, 2: 3, 6: 4, 13: 3, 14: 11, 27: 2,...",True,103.0,"{0: 5, 1: 3, 2: 3, 6: 4, 13: 3, 14: 11, 27: 2,..."


KeyError: '# of visits for visited each node'

<AxesSubplot:>

In [ ]:
PlotMazeCells(ma)
PlotMazeNodes(ma)